In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

import jqdatasdk as jq

# 请更改为你自己的用户名和密码，账号在jointquant.com上申请
jq.auth('user_name', 'password')

Exception: 认证失败;
您的账号或密码有误，请仔细核对您的账号及密码后重新登录；如您忘记密码可在聚宽官网登录页面修改：https://www.joinquant.com/user/login/index?type=login；其他问题请咨询JQData管理员，微信号：JQData02

# 动量策略

牛顿运动定律向我们揭示了速度和加速度的原理。在股价运动中，有的股虽然还在创新高，但没有加速度了，我们称之为滞涨。有的股虽然在底部，每天的涨幅看上去很小，很容易被忽略，但其实在加速。下面这两幅图，你能说出它们的相同点吗？

In [1]:
a = [x**2-3*x for x in range(-1,2)]
b = [x**2 - 3 * x for x in range(2,5)]
c = [x**2 - 3 * x for x in range(-1, 5)]

plt.figure(figsize=(9,3))
plt.subplot(131)
plt.plot(a)
plt.text(1.9,3.8, 'a')
plt.subplot(132)
plt.plot(b)
plt.text(0, 3.8, 'b')
plt.subplot(133)
plt.plot(c)
plt.text(3.8, 3.8, 'c')

NameError: name 'plt' is not defined

这两张图，虽然一个向下，一个向上，但实际上它们都是向上的抛物线，都有同样的正加速度。因为这个正加速度存在，所以图a继续运行下去，就形成了图b。图a、图b和图c其实都是同一条曲线。
这一个发现的重要性在于，图c就是老股民所说的黄金坑。现在，我们通过一个简单的参数（加速度）就可以捕捉这黄金坑了。如果前几个周期已确认能够拟合为一个向上的抛物线，当我们在股价刚刚跃过最低点，再次确认向上抛物线成立时买入，是不是就买在了起涨点上？

那么，我们又该如何发现这种黄金坑呢？这里还要用到多项式拟合。前面我们在求直线的斜率时，就是先将给定的时间序列拟合成一条直线，如果其拟合误差在可接受的范围内，我们就认为该时间序列是一条直线，并返回斜率。同样地，我们在这里，也是将给定的时间序列拟合成一条二次曲线，如果拟合误差在可接受范围内，我们就认为该时间序列是一条二次曲线，并返回系数。